In [3]:
import numpy as np
import torch
from matplotlib import pyplot as plt
import torch
import Utils
import network
from PatchesDataset import PatchesDataset
import evaluate

SyntaxError: invalid syntax (Utils.py, line 127)

In [ ]:
model = network.Conv64Features()
model.to('cpu').eval()

In [ ]:
left = Utils.get_left_image(180)
right = Utils.get_right_image(180)

In [ ]:
fig, axes = plt.subplots(3, 1, sharex=True)
fig.set_size_inches(18.5, 10.5)
axes[0].imshow(left)
axes[1].imshow(right)
axes[2].imshow(left)
axes[2].imshow(right, alpha=0.5)

In [5]:
left_model_input = evaluate.transform_image_for_model(left)
right_model_input = evaluate.transform_image_for_model(right)

In [ ]:
left_output = evaluate.transform_model_output_to_ndarray(model(left_model_input))
right_output = evaluate.transform_model_output_to_ndarray(model(right_model_input))

In [8]:
# sad napravimo petlju, koja ide od 0 do D
# lijeva slika samo jednom treba proći kroz model,
# dok ćemo desnu provlačiti D pata kroz mmodel.
# Svaki put pomaknutu za jedan piksel udesno.

In [9]:
# metoda koja prima output lijeve slike i input desne slike.
# Vraća sličnost lijeve i desne slike udaljenje za disparitet d.
def similarity_at_d(left_output, right_input, d):
    shifted_right_input = torch.from_numpy(np.roll(right_input.squeeze(0).permute(1, 2, 0).detach().numpy(), d, axis=1))
    right_output_at_d = model(shifted_right_input.permute(2, 0, 1).unsqueeze(0)).detach().squeeze(0).permute(1, 2, 0).numpy()

    return np.sum(left_output * right_output_at_d, axis=2)

In [10]:
D = 100

similarities_at_all_D_disparities = np.stack([similarity_at_d(left_output, right_input, d) for d in range(D)])

KeyboardInterrupt: 

In [ ]:
all_disp_image_slice = np.index_exp[:, 180:200, 350:450]
image_slice = np.index_exp[180:200, 350:450]

In [ ]:
similarities_at_all_D_disparities[all_disp_image_slice]

In [ ]:
predicted_disparity = np.argmin(similarities_at_all_D_disparities, axis=0).astype('uint8')

In [ ]:
real_disparity = Utils.get_disp_image(180)

In [ ]:
plt.figure(figsize=(30 ,5))
plt.imshow(left[image_slice])

In [ ]:
plt.figure(figsize=(30 ,5))
plt.imshow(right[image_slice])

In [ ]:
plt.figure(figsize=(30 ,5))
plt.imshow(real_disparity[image_slice], vmin=0, vmax=100, cmap='Greens')
plt.colorbar()

In [ ]:
plt.figure(figsize=(30 ,5))
plt.imshow(predicted_disparity[image_slice], vmin=0, vmax=100, cmap='Greens')
plt.colorbar()

In [ ]:
real_disparity[image_slice]

In [ ]:
predicted_disparity[image_slice]

In [ ]:
real_disparity.max()